In [1]:
import xarray as xr
import numpy as np
#import pandas as pd
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.optimizers import SGD
#import matplotlib.pyplot as plt

In [2]:
def openFile(var):
    return np.array(xr.open_dataset("../" + var + "_combined.nc" )[var])    

In [3]:
#Input Files
NNDT = openFile("NNDT")
NNDQ = openFile("NNDQ")

NNLHF = openFile("NNLHF") 
NNPS = openFile("NNPS") 
NNQBP = openFile("NNQBP") 
NNSHF = openFile("NNSHF") 
NNSOLIN = openFile("NNSOLIN") 
NNTBP = openFile("NNTBP") 
NNVBP = openFile("NNVBP") 

#Output Files
SPDT = openFile("SPDT")
QRL = openFile("QRL")
QRS = openFile("QRS")
SPTFLUX = openFile("SPTFLUX")

SPDQ = openFile("SPDQ")
SPQFLUX = openFile("SPQFLUX")

In [4]:
cpdair = 1.00464e3
latvap = 2.501e6
spHeat = (SPDT + QRL + QRS + SPTFLUX)*cpdair
spMoist = (SPDQ + SPQFLUX)*latvap

In [5]:
NNDT = np.moveaxis(NNDT, 1, 0)
NNDT.shape

(30, 7761, 64, 16)

In [6]:
NNDQ = np.moveaxis(NNDQ, 1, 0)
NNDQ.shape

(30, 7761, 64, 16)

In [7]:
NNLHF = np.array([NNLHF])
NNLHF.shape

(1, 7761, 64, 16)

In [8]:
NNPS = np.array([NNPS])
NNPS.shape

(1, 7761, 64, 16)

In [9]:
NNQBP = np.moveaxis(NNQBP, 1, 0)
NNQBP.shape

(30, 7761, 64, 16)

In [10]:
NNSHF = np.array([NNSHF])
NNSHF.shape

(1, 7761, 64, 16)

In [11]:
NNSOLIN = np.array([NNSOLIN])
NNSOLIN.shape

(1, 7761, 64, 16)

In [12]:
NNTBP = np.moveaxis(NNTBP, 1, 0)
NNTBP.shape

(30, 7761, 64, 16)

In [13]:
NNVBP = np.moveaxis(NNVBP, 1, 0)
NNVBP.shape

(30, 7761, 64, 16)

In [14]:
nnInput = np.concatenate((NNDT, \
                          NNDQ, \
                          NNLHF, \
                          NNPS, \
                          NNQBP, \
                          NNSHF, \
                          NNSOLIN, \
                          NNTBP, \
                          NNVBP), axis = 0)
nnInput.shape

(154, 7761, 64, 16)

In [15]:
nnInput = nnInput.ravel(order = 'F').reshape(154,-1, order = 'F')
nnInput.shape

(154, 7947264)

In [16]:
spTarget = np.concatenate((spHeat, spMoist), axis = 1)
spTarget = np.moveaxis(spTarget, 1, 0)
spTarget = spTarget.ravel(order = 'F').reshape(60,-1, order = 'F')
spTarget.shape

(60, 7947264)

In [ ]:
# ADDING INTERCEPT
nnInput = np.concatenate((nnInput, \
                               np.array([np.ones(nnInput.shape[1])])), \
                               axis = 0)

In [21]:
# TRAIN TEST SPLIT
dataShuffled = np.concatenate((nnInput, spTarget), axis = 0).transpose()
np.random.shuffle(dataShuffled)
split = .8
splitIndex = int(dataShuffled.shape[0]*split)
train = dataShuffled[0:splitIndex]
test = dataShuffled[splitIndex:len(dataShuffled)]

In [17]:
train_mu = np.mean(train, axis = 0)[:, np.newaxis]
train_min = train.min(axis = 0)[:, np.newaxis]
train_max = np.amax(train, axis = 0)[:, np.newaxis]
train_std = np.std(train, axis = 0)[:, np.newaxis]

In [18]:
train_norm = (train-train_mu)/train_std
test_norm = (test-train_mu)/train_std

In [19]:
nnInput_norm.shape

(155, 7947264)

In [20]:
spTarget.shape

(60, 7947264)

In [22]:
train.shape

(6357811, 215)

In [23]:
test.shape

(1589453, 215)

In [24]:
trainInput = train[:,0:nnInput.shape[0]]
trainOutput = train[:,nnInput.shape[0]:len(train)]
testInput = test[:,0:nnInput.shape[0]]
testOutput = test[:,nnInput.shape[0]:len(test)]

#sanity check
len(train)==len(test)

False

In [25]:
trainInput.shape

(6357811, 155)

In [26]:
trainOutput.shape

(6357811, 60)

In [27]:
testInput.shape

(1589453, 155)

In [28]:
testOutput.shape

(1589453, 60)

In [29]:
with open('trainInput.npy', 'wb') as f:
    np.save(f, trainInput)
with open('trainOutput.npy', 'wb') as f:
    np.save(f, trainOutput)
with open('testInput.npy', 'wb') as f:
    np.save(f, testInput)
with open('testOutput.npy', 'wb') as f:
    np.save(f, testOutput)

In [30]:
with open('smalltrainInput.npy', 'wb') as f:
    np.save(f, trainInput[:,0:100000])
with open('smalltrainOutput.npy', 'wb') as f:
    np.save(f, trainOutput[:,0:100000])
with open('smalltestInput.npy', 'wb') as f:
    np.save(f, testInput[:,0:100000])
with open('smalltestOutput.npy', 'wb') as f:
    np.save(f, testOutput[:,0:100000])

In [99]:
#model = Sequential()
#model.add(Dense(186, input_dim=155, kernel_initializer='normal', activation='relu'))
#model.add(Dense(217, kernel_initializer='normal', activation='relu'))
#model.add(Dense(248, kernel_initializer='normal', activation='relu'))
#model.add(Dense(201, kernel_initializer='normal', activation='relu'))
#model.add(Dense(154, kernel_initializer='normal', activation='relu'))
#model.add(Dense(107, kernel_initializer='normal', activation='relu'))
#model.add(Dense(60, activation='linear'))



In [100]:
#model.compile(optimizer='adam', loss='mse')

In [101]:
#trainFit = model.fit(trainInput, trainOutput, batch_size=1563, epochs=100)